In [160]:
"""
@author: pho
"""
%load_ext autoreload
%autoreload 2
import sys
import importlib

import numpy as np
import pandas as pd

## Panel:
import param
import panel as pn
from panel.viewable import Viewer

## Pho's Custom Libraries:
from pyphocorehelpers.general_helpers import PrettyPrintable, get_arguments_as_optional_dict
from pyphocorehelpers.function_helpers import compose_functions
from pyphocorehelpers.indexing_helpers import partition, build_spanning_bins

from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import * # get_neuron_identities
from pyphoplacecellanalysis.General.SessionSelectionAndFiltering import batch_filter_session
from pyphoplacecellanalysis.General.ComputationResults import ComputationResult
# from PendingNotebookCode import estimation_session_laps


from neuropy.analyses.laps import estimation_session_laps
from neuropy.core.epoch import NamedTimerange

# Neuropy:
from neuropy.analyses.placefields import PlacefieldComputationParameters, perform_compute_placefields
from neuropy.core.neuron_identities import NeuronIdentity, build_units_colormap, PlotStringBrevityModeEnum
from neuropy.utils.debug_helpers import debug_print_placefield, debug_print_spike_counts, debug_print_subsession_neuron_differences

# def estimate_session_laps_load_function(regular_load_function, a_base_dir):
#     session = regular_load_function(a_base_dir)
#     ## Estimate the Session's Laps data using my algorithm from the loaded position data.
#     session = estimation_session_laps(session)
#     return session

known_data_session_type_dict = {'kdiba':KnownDataSessionTypeProperties(load_function=(lambda a_base_dir: DataSessionLoader.kdiba_old_format_session(a_base_dir)),
                               basedir=Path(r'R:\data\KDIBA\gor01\one\2006-6-07_11-26-53')),
                'bapun':KnownDataSessionTypeProperties(load_function=(lambda a_base_dir: DataSessionLoader.bapun_data_session(a_base_dir)),
                               basedir=Path('R:\data\Bapun\Day5TwoNovel'))
               }

known_data_session_type_dict['kdiba'].post_load_functions = [lambda a_loaded_sess: estimation_session_laps(a_loaded_sess)]
# known_data_session_type_dict = {'kdiba':KnownDataSessionTypeProperties(load_function=(lambda a_base_dir: estimation_session_laps(DataSessionLoader.kdiba_old_format_session(a_base_dir))),
#                                basedir=Path(r'R:\data\KDIBA\gor01\one\2006-6-07_11-26-53')),
#                 'bapun':KnownDataSessionTypeProperties(load_function=(lambda a_base_dir: DataSessionLoader.bapun_data_session(a_base_dir)),
#                                basedir=Path('R:\data\Bapun\Day5TwoNovel'))
#                }

# known_data_session_type_dict['kdiba'].name

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Truely Common:
def _display_result(computation_result):
    pf_neuron_identities, pf_sort_ind, pf_colors, pf_colormap, pf_listed_colormap = get_neuron_identities(computation_result.computed_data['pf2D'])
    computation_result.computed_data['pf2D'].plot_raw(label_cells=True); # Plots an overview of each cell all in one figure
    computation_result.computed_data['pf2D'].plot_ratemaps_2D(resolution_multiplier=2.5, brev_mode=PlotStringBrevityModeEnum.MINIMAL)

    

# Bapun Format:

In [ ]:
# curr_bapun_pipeline = NeuropyPipeline(name='bapun_pipeline', session_data_type='bapun', basedir=known_data_session_type_dict['bapun'].basedir, load_function=known_data_session_type_dict['bapun'].load_function)
curr_bapun_pipeline = NeuropyPipeline.init_from_known_data_session_type('bapun', known_data_session_type_dict['bapun'])

# curr_bapun_pipeline = NeuropyPipeline.init_from_known_data_session_type('bapun', known_data_session_type_dict['bapun'])
curr_bapun_pipeline.is_loaded
size_bytes = curr_bapun_pipeline.sess.__sizeof__() # 1753723032
f'object size: {size_bytes/(1024*1024)} MB'

In [ ]:
# Bapun/DataFrame style session filter functions:
def _temp_filter_session_by_epoch1(sess):
    """ 
    Usage:
        active_session, active_epoch = _temp_filter_session(curr_bapun_pipeline.sess)
    """
    active_epoch = sess.epochs.get_named_timerange('maze1')
    ## All Spikes:
    # active_epoch_session = sess.filtered_by_epoch(active_epoch) # old
    active_session = batch_filter_session(sess, sess.position, sess.spikes_df, active_epoch.to_Epoch())
    return active_session, active_epoch

def _temp_filter_session_by_epoch2(sess):
    """ 
    Usage:
        active_session, active_epoch = _temp_filter_session(curr_bapun_pipeline.sess)
    """
    active_epoch = sess.epochs.get_named_timerange('maze2')
    ## All Spikes:
    # active_epoch_session = sess.filtered_by_epoch(active_epoch) # old
    active_session = batch_filter_session(sess, sess.position, sess.spikes_df, active_epoch.to_Epoch())
    return active_session, active_epoch


active_session_filter_configurations = {'maze1':_temp_filter_session_by_epoch1,
                                        'maze2':_temp_filter_session_by_epoch2
                                       }

curr_bapun_pipeline.filter_sessions(active_session_filter_configurations)
curr_bapun_pipeline.perform_computations(PlacefieldComputationParameters(speed_thresh=0.0, grid_bin=(5, 3), smooth=(0.0, 0.0), frate_thresh=0.1))

In [ ]:
_display_result(curr_bapun_pipeline.computation_results['maze1'])

In [ ]:
_display_result(curr_bapun_pipeline.computation_results['maze2'])

# KDiba Format:

In [3]:
## Data must be pre-processed using the MATLAB script located here: 
# R:\data\KDIBA\gor01\one\IIDataMat_Export_ToPython_2021_11_23.m
# From pre-computed .mat files:
## 07: 
# basedir = r'R:\data\KDIBA\gor01\one\2006-6-07_11-26-53'
# # ## 08:
# basedir = r'R:\data\KDIBA\gor01\one\2006-6-08_14-26-15'
# curr_kdiba_pipeline = NeuropyPipeline(name='kdiba_pipeline', session_data_type='kdiba', basedir=known_data_session_type_dict['kdiba'].basedir, load_function=known_data_session_type_dict['kdiba'].load_function)
curr_kdiba_pipeline = NeuropyPipeline.init_from_known_data_session_type('kdiba', known_data_session_type_dict['kdiba'])

# curr_bapun_pipeline
curr_kdiba_pipeline.is_loaded
size_bytes = curr_kdiba_pipeline.sess.__sizeof__() # 1753723032
f'object size: {size_bytes/(1024*1024)} MB'
# ## Estimate the Session's Laps data using my algorithm from the loaded position data.
# curr_kdiba_pipeline.sess = estimation_session_laps(curr_kdiba_pipeline.sess)
curr_kdiba_pipeline.sess.epochs

basedir is already Path object.
	 basepath: R:\data\KDIBA\gor01\one\2006-6-07_11-26-53
	 session_name: 2006-6-07_11-26-53
Loading matlab import file results to R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.epochs_info.mat... done.
Loading matlab import file results to R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.position_info.mat... done.
Loading matlab import file results to R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.spikes.mat... done.
Failure loading .position.npy. Must recompute.

Computing linear positions for all active epochs for session... Saving updated position results results to R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.position.npy... 2006-6-07_11-26-53.position.npy saved
done.
	 Failure loading .interpolated_spike_positions.npy. Must recompute.

	 Saving updated interpolated spike position results results to R:\data\KDIBA\gor01\one\2006-6-07_11-26-53\2006-6-07_11-26-53.interpolated_spike_positions.npy.

         start         stop  label     duration
0     0.000000  1739.153364  maze1  1739.153364
1  1739.153364  1932.420005  maze2   193.266641

In [ ]:
def _temp_filter_session_by_epoch1(sess):
    """ 
    Usage:
        active_session, active_epoch = _temp_filter_session(curr_bapun_pipeline.sess)
    """
    active_epoch = sess.epochs.get_named_timerange('maze1')
    ## All Spikes:
    active_session = sess.filtered_by_epoch(active_epoch) # kdiba
    # active_session = batch_filter_session(sess, sess.position, sess.spikes_df, active_epoch.to_Epoch())
    return active_session, active_epoch

def _temp_filter_session_by_epoch2(sess):
    """ 
    Usage:
        active_session, active_epoch = _temp_filter_session(curr_bapun_pipeline.sess)
    """
    active_epoch = sess.epochs.get_named_timerange('maze2')
    ## All Spikes:
    active_session = sess.filtered_by_epoch(active_epoch) # kdiba
    # active_session = batch_filter_session(sess, sess.position, sess.spikes_df, active_epoch.to_Epoch()) # new Bapun/Df
    return active_session, active_epoch

active_session_filter_configurations = {'maze1':_temp_filter_session_by_epoch1,
                                        'maze2':_temp_filter_session_by_epoch2
                                       }

# active_config = build_configs(sess.config, active_epoch)

curr_kdiba_pipeline.filter_sessions(active_session_filter_configurations)
curr_kdiba_pipeline.perform_computations(PlacefieldComputationParameters(speed_thresh=0.0, grid_bin=(5, 3), smooth=(0.0, 0.0), frate_thresh=0.1))

In [143]:
def build_lap_epochs_filters(sess):
    lap_specific_epochs = sess.laps.as_epoch_obj()
    # lap_specific_epochs.to_dataframe()
    any_lap_specific_epochs = lap_specific_epochs.label_slice(lap_specific_epochs.labels[np.arange(len(sess.laps.lap_id))])
    even_lap_specific_epochs = lap_specific_epochs.label_slice(lap_specific_epochs.labels[np.arange(0, len(sess.laps.lap_id), 2)])
    odd_lap_specific_epochs = lap_specific_epochs.label_slice(lap_specific_epochs.labels[np.arange(1, len(sess.laps.lap_id), 2)])
    
    ## All Spikes:
    sess.epochs.t_start = 22.26 # exclude the first short period where the animal isn't on the maze yet
    # sess.epochs.to_dataframe()
    # active_epoch = sess.epochs.get_named_timerange('maze1')
    # print('active_epoch: {}'.format(active_epoch))
    # active_epoch = sess.epochs.get_named_timerange('maze2')
    # active_epoch_maze_all = NamedTimerange(name='maze', start_end_times=[sess.epochs['maze1'][0], sess.epochs['maze2'][1]])

    # active_epoch_session = sess.filtered_by_neuron_type('pyramidal').filtered_by_epoch(active_epoch)
    # print_subsession_neuron_differences(sess.neurons, active_epoch_session.neurons)

    active_session_filter_configurations = {'maze1': lambda x: (x.filtered_by_neuron_type('pyramidal').filtered_by_epoch(x.epochs.get_named_timerange('maze1')), x.epochs.get_named_timerange('maze1')),
                                        'maze2': lambda x: (x.filtered_by_neuron_type('pyramidal').filtered_by_epoch(x.epochs.get_named_timerange('maze2')), x.epochs.get_named_timerange('maze2')),
                                        'maze': lambda x: (x.filtered_by_neuron_type('pyramidal').filtered_by_epoch(NamedTimerange(name='maze', start_end_times=[x.epochs['maze1'][0], x.epochs['maze2'][1]])), NamedTimerange(name='maze', start_end_times=[x.epochs['maze1'][0], x.epochs['maze2'][1]]))
                                       }
    return active_session_filter_configurations
    
active_session_filter_configurations = build_lap_epochs_filters(curr_kdiba_pipeline.sess)
curr_kdiba_pipeline.filter_sessions(active_session_filter_configurations)
curr_kdiba_pipeline.perform_computations(PlacefieldComputationParameters(speed_thresh=0.0, grid_bin=(5, 3), smooth=(0.0, 0.0), frate_thresh=0.1))

In [158]:
## Position Decoding:

## Stock Decoder:
from neuropy.analyses.decoders import Decode1d

curr_result_label = 'maze1'
sess = curr_kdiba_pipeline.filtered_sessions[curr_result_label]
pf = curr_kdiba_pipeline.computation_results[curr_result_label].computed_data['pf1D']
maze1 = sess.paradigm[curr_result_label]
# rpls = sess.ripple.time_slice(maze1[0], maze1[1])
rpls = None
pf_neurons = sess.neurons.get_by_id(pf.ratemap.neuron_ids) 
decode = Decode1d(neurons=pf_neurons, ratemap = pf.ratemap, epochs=rpls, bin_size=0.02)
decode

# Plot to validate decoder:
np.shape(decode.decoded_position) # (85845,)
plt.plot(decode.decoded_position)
ax = plt.gca()
# ax.xlim() # (-4292.2, 90136.2)
ax.set_xlim(10000, 12000)

np.shape(decode.posterior) # (48, 85845)

In [202]:
# My custom Decoder
time_bin_size = 0.02

curr_result_label = 'maze1'
sess = curr_kdiba_pipeline.filtered_sessions[curr_result_label]
pf = curr_kdiba_pipeline.computation_results[curr_result_label].computed_data['pf1D']

spikes_df = sess.spikes_df
time_variable_name = spikes_df.spikes.time_variable_name # 't_rel_seconds'
# time_bin_size

out_digitized_variable_bins, out_binning_info = build_spanning_bins(spikes_df[time_variable_name].to_numpy(), max_bin_size=time_bin_size, debug_print=False) # np.shape(out_digitized_variable_bins)[0] == np.shape(spikes_df)[0]
print(f'spikes_df[time_variable_name]: {np.shape(spikes_df[time_variable_name])}\nout_digitized_variable_bins: {np.shape(out_digitized_variable_bins)}')
# assert (np.shape(out_digitized_variable_bins)[0] == np.shape(spikes_df)[0]), f'np.shape(out_digitized_variable_bins)[0]: {np.shape(out_digitized_variable_bins)[0]} should equal np.shape(spikes_df)[0]: {np.shape(spikes_df)[0]}'
print(out_binning_info)
spikes_df

spikes_df[time_variable_name]: (69142,)
out_digitized_variable_bins: (85842,)
BinningInfo(variable_extents=(22.30206346133491, 1739.1355703625595), step=0.019999924359884725, num_bins=85842, bin_indicies=array([    0,     1,     2, ..., 85839, 85840, 85841]))


,t,t_seconds,t_rel_seconds,shank,cluster,aclu,qclu,x,y,speed,...,lap,maze_relative_lap,maze_id,cell_type,flat_spike_idx,x_loaded,y_loaded,lin_pos,unit_id,binned_time
12517,725979.0,506939.938112,22.302063,7,7,42,4,228.439138,145.608734,4.518929,...,-1,-1,1,NeuronType.PYRAMIDAL,12517,0.793762,0.498754,-46.025885,40,NaN
12520,726167.0,506939.943888,22.307839,2,5,14,4,228.428329,145.632192,4.517234,...,-1,-1,1,NeuronType.PYRAMIDAL,12520,0.793826,0.498658,-46.015141,12,"(22.302, 22.322]"
12532,726835.0,506939.964409,22.328360,2,28,27,4,228.389921,145.715544,4.511208,...,-1,-1,1,NeuronType.PYRAMIDAL,12532,0.794084,0.498267,-45.976965,25,"(22.322, 22.342]"
12627,731665.0,506940.112786,22.476737,11,2,58,4,228.110399,146.322156,4.495592,...,-1,-1,1,NeuronType.PYRAMIDAL,12627,0.795136,0.497673,-45.699695,56,"(22.462, 22.482]"
12633,732082.0,506940.125597,22.489548,8,13,53,4,228.085786,146.375570,4.484851,...,-1,-1,1,NeuronType.PYRAMIDAL,12633,0.795016,0.497707,-45.675124,51,"(22.482, 22.502]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831590,56605920.0,508656.569021,1738.932972,2,20,21,4,212.373363,138.488006,15.584255,...,-1,-1,1,NeuronType.PYRAMIDAL,831590,0.766190,0.478007,-29.938979,19,"(1738.916, 1738.936]"
831601,56607201.0,508656.608373,1738.972324,2,20,21,4,211.783865,138.621493,15.438952,...,-1,-1,1,NeuronType.PYRAMIDAL,831601,0.763523,0.477928,-29.351434,19,"(1738.956, 1738.976]"
831623,56609125.0,508656.667479,1739.031430,11,2,58,4,210.881567,138.825812,15.409233,...,-1,-1,1,NeuronType.PYRAMIDAL,831623,0.759478,0.477808,-28.475510,56,"(1739.016, 1739.036]"
831662,56612413.0,508656.768486,1739.132437,2,20,21,4,209.352483,139.172060,15.495960,...,-1,-1,1,NeuronType.PYRAMIDAL,831662,0.752613,0.477605,-27.286186,19,"(1739.116, 1739.136]"


In [203]:
spikes_df[time_variable_name].value_counts(bins=out_binning_info.num_bins, sort=False)

(20.584, 22.322]        2
(22.322, 22.342]        1
(22.342, 22.362]        0
(22.362, 22.382]        0
(22.382, 22.402]        0
                       ..
(1739.036, 1739.056]    0
(1739.056, 1739.076]    0
(1739.076, 1739.096]    0
(1739.096, 1739.116]    0
(1739.116, 1739.136]    2
Name: t_rel_seconds, Length: 85842, dtype: int64

In [209]:
# cut_bins = np.linspace(59200, 60800, 9)
# pd.cut(df[time_variable_name], bins=cut_bins)

# pd.cut(df['ext price'], bins=4)
# time_binned_spikes_df = pd.cut(spikes_df[time_variable_name].to_numpy(), bins=out_digitized_variable_bins)
spikes_df['binned_time'] = pd.cut(spikes_df[time_variable_name].to_numpy(), bins=out_digitized_variable_bins, include_lowest=True, labels=out_binning_info.bin_indicies[1:]) # same shape as the input data (time_binned_spikes_df: (69142,))
# print(f'time_binned_spikes_df: {np.shape(time_binned_spikes_df)}, type(time_binned_spikes_df): {type(time_binned_spikes_df)}')
# spikes_df['binned_time'] = time_binned_spikes_df
## NOTE: type(time_binned_spikes_df) == tuple!

# spikes_df[time_variable_name].to_numpy()[time_binned_spikes_df]

spikes_df

time_binned_spikes_df: (69142,), type(time_binned_spikes_df): <class 'pandas.core.arrays.categorical.Categorical'>


,t,t_seconds,t_rel_seconds,shank,cluster,aclu,qclu,x,y,speed,...,lap,maze_relative_lap,maze_id,cell_type,flat_spike_idx,x_loaded,y_loaded,lin_pos,unit_id,binned_time
12517,725979.0,506939.938112,22.302063,7,7,42,4,228.439138,145.608734,4.518929,...,-1,-1,1,NeuronType.PYRAMIDAL,12517,0.793762,0.498754,-46.025885,40,1
12520,726167.0,506939.943888,22.307839,2,5,14,4,228.428329,145.632192,4.517234,...,-1,-1,1,NeuronType.PYRAMIDAL,12520,0.793826,0.498658,-46.015141,12,1
12532,726835.0,506939.964409,22.328360,2,28,27,4,228.389921,145.715544,4.511208,...,-1,-1,1,NeuronType.PYRAMIDAL,12532,0.794084,0.498267,-45.976965,25,2
12627,731665.0,506940.112786,22.476737,11,2,58,4,228.110399,146.322156,4.495592,...,-1,-1,1,NeuronType.PYRAMIDAL,12627,0.795136,0.497673,-45.699695,56,9
12633,732082.0,506940.125597,22.489548,8,13,53,4,228.085786,146.375570,4.484851,...,-1,-1,1,NeuronType.PYRAMIDAL,12633,0.795016,0.497707,-45.675124,51,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831590,56605920.0,508656.569021,1738.932972,2,20,21,4,212.373363,138.488006,15.584255,...,-1,-1,1,NeuronType.PYRAMIDAL,831590,0.766190,0.478007,-29.938979,19,85831
831601,56607201.0,508656.608373,1738.972324,2,20,21,4,211.783865,138.621493,15.438952,...,-1,-1,1,NeuronType.PYRAMIDAL,831601,0.763523,0.477928,-29.351434,19,85833
831623,56609125.0,508656.667479,1739.031430,11,2,58,4,210.881567,138.825812,15.409233,...,-1,-1,1,NeuronType.PYRAMIDAL,831623,0.759478,0.477808,-28.475510,56,85836
831662,56612413.0,508656.768486,1739.132437,2,20,21,4,209.352483,139.172060,15.495960,...,-1,-1,1,NeuronType.PYRAMIDAL,831662,0.752613,0.477605,-27.286186,19,85841


In [201]:
# type(spikes_df['binned_time'].values) # pandas.core.arrays.categorical.Categorical
# spikes_df['binned_time'].values.categories
spikes_df['binned_time'].values.astype(str)
# time_binned_spikes_df[1]

array(['nan', '(22.302, 22.322]', '(22.322, 22.342]', ...,
       '(1739.016, 1739.036]', '(1739.116, 1739.136]',
       '(1739.116, 1739.136]'], dtype=object)

In [236]:
# df.groupby(bins)['Value'].agg(['count', 'sum'])
# spikes_df.groupby(time_binned_spikes_df)[time_variable_name].agg('count')

any_unit_spike_counts = spikes_df.groupby(['binned_time'])[time_variable_name].agg('count')

unit_specific_bin_specific_spike_counts = spikes_df.groupby(['aclu','binned_time'])[time_variable_name].agg('count')
unit_specific_bin_specific_spike_counts

aclu  binned_time
2     1              0
      2              0
      3              0
      4              0
      5              0
                    ..
64    85837          0
      85838          0
      85839          0
      85840          0
      85841          0
Name: t_rel_seconds, Length: 3433640, dtype: int64

In [237]:
test_multiindex = unit_specific_bin_specific_spike_counts.index
active_aclu_values = np.unique(test_multiindex.get_level_values('aclu'))

# test_multiindex.get_level_values('aclu')
# test_multiindex.get_level_values('binned_time')

In [246]:
unit_specific_spike_counts = np.array([unit_specific_bin_specific_spike_counts[aclu].values for aclu in active_aclu_values]).T # (85841, 40)
unit_specific_spike_counts

print(f'np.shape(unit_specific_spike_counts): {np.shape(unit_specific_spike_counts)}') # np.shape(unit_specific_spike_counts): (40, 85841)
# np.array(

# unit_specific_spike_counts.get_group(2)

# spikes_df.groupby(['binned_time']).agg('count')

# for name, group in spikes_df.groupby(['aclu','binned_time']):
#     print(f'name: {name}, group: {group}') 
    
# neuron_ids, neuron_specific_spikes_dfs = partition(spikes_df, 'aclu')


    
# spikes_df.groupby(['aclu','binned_time'])

# .unstack()

# groups.size().unstack()

# spikes_df._obj.groupby(['aclu'])
# neuron_ids, neuron_specific_spikes_dfs = partition(spikes_df, 'aclu')


np.shape(unit_specific_spike_counts): (85841, 40)


120

In [ ]:
def debug_plot_filtered_subsession_neuron_differences(sess, filtered_sess):
    print_subsession_neuron_differences(sess.neurons, active_epoch_session.neurons)

[debug_plot_filtered_subsession_neuron_differences(curr_kdiba_pipeline.sess, a_filtered_sess) for a_filtered_sess in curr_kdiba_pipeline.filtered_sessions]

In [ ]:
curr_kdiba_pipeline.computation_results['maze2']

In [ ]:
_display_result(curr_kdiba_pipeline.computation_results['maze1'])
_display_result(curr_kdiba_pipeline.computation_results['maze2'])

In [ ]:


def _display_normal(computation_result, active_config):
    from neuropy.plotting.placemaps import plot_all_placefields
    # from neuropy.core.neuron_identities import build_units_colormap
    # print(f'active_config: {active_config}')
    # active_config = computation_result.sess.config
    if active_config.computation_config is None:
        active_config.computation_config = computation_result.computation_config

    pf_neuron_identities, pf_sort_ind, pf_colors, pf_colormap, pf_listed_colormap = get_neuron_identities(computation_result.computed_data['pf2D'])
    ax_pf_1D, occupancy_fig, active_pf_2D_figures, active_pf_2D_gs = plot_all_placefields(computation_result.computed_data['pf1D'], computation_result.computed_data['pf2D'], active_config, should_save_to_disk=False)
    
# curr_kdiba_pipeline.computation_results['maze1'].computation_config
# curr_kdiba_pipeline.computation_results['maze1'].sess.config
# curr_kdiba_pipeline.active_configs['maze1']
_display_normal(curr_kdiba_pipeline.computation_results['maze1'], curr_kdiba_pipeline.active_configs['maze1'])

In [139]:
import matplotlib.pyplot as plt
from neuropy.utils.misc import is_iterable
from neuropy.plotting.figure import pretty_plot
from scipy.ndimage import gaussian_filter, gaussian_filter1d, interpolation

from pyphoplacecellanalysis.Analysis.reliability import compute_lap_to_lap_reliability

from PhoPositionalData.plotting.laps import plot_laps_2d


def _test_plotRaw_v_time(active_pf, cellind, speed_thresh=False, alpha=0.5, ax=None):
        """ Builds one subplot for each dimension of the position data
        
        Updated to work with both 1D and 2D Placefields """   
        if ax is None:
            fig, ax = plt.subplots(active_pf.ndim, 1, sharex=True)
            fig.set_size_inches([23, 9.7])
        
        if not is_iterable(ax):
            ax = [ax]
            
        # plot trajectories
        if active_pf.ndim < 2:
            variable_array = [active_pf.x]
            label_array = ["X position (cm)"]
        else:
            variable_array = [active_pf.x, active_pf.y]
            label_array = ["X position (cm)", "Y position (cm)"]
            
        for a, pos, ylabel in zip(ax, variable_array, label_array):
            a.plot(active_pf.t, pos)
            a.set_xlabel("Time (seconds)")
            a.set_ylabel(ylabel)
            pretty_plot(a)

        # Grab correct spike times/positions
        if speed_thresh:
            spk_pos_, spk_t_ = active_pf.run_spk_pos, active_pf.run_spk_t
        else:
            spk_pos_, spk_t_ = active_pf.spk_pos, active_pf.spk_t

        # plot spikes on trajectory
        for a, pos in zip(ax, spk_pos_[cellind]):
            a.plot(spk_t_[cellind], pos, ".", color=[0, 0, 0.8, alpha])

        # Put info on title
        ax[0].set_title(
            "Cell "
            + str(active_pf.cell_ids[cellind])
            + ":, speed_thresh="
            + str(active_pf.speed_thresh)
        )
        return ax


def compute_reliability_metrics(out_indicies, out_digitized_position_bins, out_within_lap_spikes_overlap, debug_print=False, plot_results=False):
    """ Takes input from compute_lap_to_lap_reliability(...) to build the actual reliability metrics """
    # Actual Computations of Reliability:
    out_pairwise_pair_results = np.zeros_like(out_within_lap_spikes_overlap)
    
    # do simple diff:
    laps_spikes_overlap_diff = np.diff(out_within_lap_spikes_overlap, axis=1) # the element-wise diff of the overlap. Shows changes.
    out_pairwise_pair_results[:, 1:] = laps_spikes_overlap_diff
    # out_pairwise_pair_results[:, -1] = np.zeros_like(out_within_lap_spikes_overlap[:,0])
    
    # do custom pairwise operation:
#     for first_item_lap_idx, next_item_lap_idx in list(out_pairwise_flat_lap_indicies):
#         first_item = out_within_lap_spikes_overlap[:, first_item_lap_idx]
#         next_item = out_within_lap_spikes_overlap[:, next_item_lap_idx]
#         out_pairwise_pair_results[:, next_item_lap_idx] = (first_item * next_item) # the result should be stored in the index of the second item, if we're doing the typical backwards style differences.
#         # print(f'np.max(out_pairwise_pair_results[:, next_item_lap_idx]): {np.max(out_pairwise_pair_results[:, next_item_lap_idx])}')

    if debug_print: 
        print(f'max out: {np.max(out_pairwise_pair_results)}')
        
    lap_ids 
    flat_lap_idxs = np.arange(len(lap_ids))
    
    
    # add to the extant plot as a new color:
    if plot_results:
        for lap_idx, lap_ID in zip(flat_lap_idxs, lap_ids):
            # curr_lap_alt_ax = axs[lap_idx]
            if plot_horizontal:
                curr_lap_alt_ax = axs[lap_idx].twiny()
                curr_lap_alt_ax.plot(out_pairwise_pair_results[:, lap_idx], out_digitized_position_bins, '--r')
            else:
                # vertical
                curr_lap_alt_ax = axs[lap_idx].twinx()
                curr_lap_alt_ax.plot(out_digitized_position_bins, out_pairwise_pair_results[:, lap_idx], '--r')
            
    cum_laps_reliability = np.cumprod(out_within_lap_spikes_overlap, axis=1)
    all_laps_reliability = np.prod(out_within_lap_spikes_overlap, axis=1, keepdims=True)
    
    if plot_results:
        fig_result, axs_result = plt.subplots(2, 1, sharex=True, sharey=True, figsize=(24, 40))
        axs_result[0].plot(out_digitized_position_bins, all_laps_reliability, 'r')
        axs_result[1].plot(out_digitized_position_bins, cum_laps_reliability, 'r')

curr_result_label = 'maze1'
sess = curr_kdiba_pipeline.filtered_sessions[curr_result_label]
sess = curr_kdiba_pipeline.sess

In [ ]:
curr_laps_df = sess.laps.to_dataframe()
curr_laps_df

In [ ]:
pos_df = sess.compute_position_laps() # ensures the laps are computed if they need to be:
position_obj = sess.position
position_obj.compute_higher_order_derivatives()
pos_df = position_obj.compute_smoothed_position_info(N=20) ## Smooth the velocity curve to apply meaningful logic to it
pos_df = position_obj.to_dataframe()
pos_df

In [ ]:
# fig, out_axes_list = plot_laps_2d(sess, legacy_plotting_mode=True)
fig, out_axes_list = plot_laps_2d(sess, legacy_plotting_mode=False)
out_axes_list[0].set_title('New Pho Position Thresholding Estimated Laps')

curr_cell_idx = 2 
# curr_cell_idx = 3 # good for end platform analysis
curr_cell_ID = sess.spikes_df.spikes.neuron_ids[curr_cell_idx]
print(f'curr_cell_idx: {curr_cell_idx}, curr_cell_ID: {curr_cell_ID}')

# pre-filter by spikes that occur in one of the included laps for the filtered_spikes_df
filtered_spikes_df = sess.spikes_df.copy()
time_variable_name = filtered_spikes_df.spikes.time_variable_name # 't_rel_seconds'

lap_ids = sess.laps.lap_id
# lap_flat_idxs = sess.laps.get_lap_flat_indicies(lap_ids)

out_indicies, out_digitized_position_bins, out_within_lap_spikes_overlap = compute_lap_to_lap_reliability(curr_kdiba_pipeline.computation_results[curr_result_label].computed_data['pf2D'], filtered_spikes_df, lap_ids, curr_cell_idx, debug_print=False, plot_results=True);

# compute_reliability_metrics(out_indicies, out_digitized_position_bins, out_within_lap_spikes_overlap, debug_print=False, plot_results=False)

# # curr_kdiba_pipeline.computation_results['maze1'].computed_data['pf2D'].plotRaw_v_time(curr_cell_idx)
# _test_plotRaw_v_time(curr_kdiba_pipeline.computation_results[curr_result_label].computed_data['pf2D'], curr_cell_idx)

# 3D Lap Plotting Experimentation

In [ ]:
from itertools import islice # for Pagination class
import pyvista as pv
import pyvistaqt as pvqt
from PhoGui.InteractivePlotter.LapsVisualizationMixin import LapsVisualizationMixin
from PhoGui.PhoCustomVtkWidgets import PhoWidgetHelper
from PhoPositionalData.plotting.spikeAndPositions import perform_plot_flat_arena, _build_flat_arena_data

""" Test Drawing Spike Lines """
# from pyphoplacecellanalysis.Pho3D.spikes import draw_line_spike, lines_from_points
from pyphoplacecellanalysis.Pho3D.points import interlieve_points

from PhoPositionalData.plotting.spikeAndPositions import build_active_spikes_plot_pointdata_df

def _plot_all_lap_spikes(p, sess, included_cell_IDXs, included_lap_IDXs, debug_print=True, lap_start_z=0.0, lap_id_dependent_z_offset=10.0):
    should_reinterpolate_spike_positions = False
        
    def _plot_single_spikes(p, cell_specific_spikes_dfs, placefield_cell_index):
        curr_cell_spike_df = cell_specific_spikes_dfs[placefield_cell_index]
        # curr_cell_spike_df['z_fixed'] = np.full_like(active_flat_df['x'].values, 1.1)
        pdata = build_active_spikes_plot_pointdata_df(curr_cell_spike_df)

        # curr_cell_spike_times = curr_cell_spike_df[curr_cell_spike_df.spikes.time_variable_name].to_numpy()  # (271,)
        # curr_cell_spike_positions = curr_cell_spike_df['x','y'].to_numpy()  # (271,)

        # lines_from_points(
        # p[0,0].add_points(pdata, name='plot_single_spikes_points', render_points_as_spheres=True, point_size=5.0)

        # Build offset points and spike data:
        spike_height = max((lap_id_dependent_z_offset * 0.6), 0.5) # half the line height

        # start_points = pdata.points.copy()
        # end_points = start_points.copy()
        # # end_points[:,2] # get z values
        # end_points[:,2] = end_points[:,2] + spike_height
        # all_points = interlieve_points(start_points, end_points)
        # lines_poly_data = pv.PolyData()
        # lines_poly_data.points = all_points
        # # cells = np.hstack(([2, 0, 1],[2, 1, 2]))
        # num_lines = np.shape(start_points)[0]
        # cells = [[2, 2*i, 2*i+1] for i in np.arange(num_lines)]
        # lines_poly_data.lines = cells
        
        p[0,0].add_mesh(pdata, name=f'plot_single_spikes_points[{placefield_cell_index}]', render_points_as_spheres=True, point_size=5.0, color='white')        
        
        # p[0,0].add_points(pdata, name=f'plot_single_spikes_points[{placefield_cell_index}]', render_points_as_spheres=True, point_size=5.0)
        # p[0,0].add_mesh(lines_poly_data, name=f'plot_single_spikes_lines[{placefield_cell_index}]', render_points_as_spheres=False, point_size=5.0)
        # return {'pdata':pdata, 'lines_poly_data':lines_poly_data}
        
        return {'pdata':pdata}
    
    time_variable_name = sess.spikes_df.spikes.time_variable_name # 't_rel_seconds'
    # sets the 'z_fixed' value for all spikes in sess.spikes_df, which will be used to plot them as points
    sess.spikes_df['z'] = lap_start_z + (lap_id_dependent_z_offset * sess.spikes_df.lap.to_numpy())

    if debug_print:
        print(f'sess.laps.lap_id: {sess.laps.lap_id}')
        
    included_cell_IDXs = np.array(included_cell_IDXs)
    included_lap_IDXs = np.array(included_lap_IDXs)
    
    # ensure that only lap_ids included in this session are used:
    included_lap_ids = sess.laps.lap_id[included_lap_IDXs]
    possible_included_lap_ids = np.unique(sess.spikes_df.lap.values)
    if debug_print:
        print(f'np.unique(sess.spikes_df.lap.values): {np.unique(sess.spikes_df.lap.values)}')
    included_lap_ids = included_lap_ids[np.isin(included_lap_ids, possible_included_lap_ids)]
    if debug_print:
        print(f'included_lap_ids: {included_lap_ids}')
    
    # get the included cell IDs
    included_cell_IDs = np.array(sess.spikes_df.spikes.neuron_ids)[included_cell_IDXs]
        
    # print(np.isin(['R','G','B','render_opacity'], sess.spikes_df.columns).all())

    # POSITIONS:
    curr_position_df, lap_specific_position_dfs, lap_specific_time_ranges, lap_specific_position_traces = LapsVisualizationMixin._compute_laps_position_data(sess)
    
    # SPIKES:
    # # grouped by lap
    # lap_grouped_spikes_df = sess.spikes_df.groupby('lap')
    # lap_specific_spikes_dfs = [lap_grouped_spikes_df.get_group(i)[[time_variable_name,'aclu','lap','flat_spike_idx','cell_type','x','y','lin_pos','z']] for i in included_lap_ids] # dataframes split for each ID:

    # grouped by cell:
    # pre-filter by spikes that occur in one of the included laps for the filtered_spikes_df
    filtered_spikes_df = sess.spikes_df.copy()
    filtered_spikes_df = filtered_spikes_df[np.isin(filtered_spikes_df['lap'], included_lap_ids)] # get only the spikes that occur in one of the included laps for the filtered_spikes_df
    
    
    # Interpolate the spikes positions again:
    if should_reinterpolate_spike_positions:
        print('Re-interpolating spike positions...')
        filtered_spikes_df = filtered_spikes_df.spikes.interpolate_spike_positions(curr_position_df['t'].to_numpy(), curr_position_df['x'].to_numpy(), curr_position_df['y'].to_numpy())
        # filtered_spikes_df = FlattenedSpiketrains.interpolate_spike_positions(filtered_spikes_df, session.position.time, session.position.x, session.position.y, spike_timestamp_column_name=time_variable_name)
    
    cell_grouped_spikes_df = filtered_spikes_df.groupby('aclu')
    cell_specific_spikes_dfs = [cell_grouped_spikes_df.get_group(i)[[time_variable_name,'aclu','lap','flat_spike_idx','cell_type','x','y','lin_pos','z']] for i in included_cell_IDs] # dataframes split for each ID:

    # lap_specific_position_dfs = _compute_laps_position_data(sess)

    # # Positions:
    # curr_position_df = sess.compute_position_laps()
    # included_pos_lap_ids = np.unique(curr_position_df.lap.values)
    # print(f'np.unique(curr_position_df.lap.values): {np.unique(curr_position_df.lap.values)}')
    # included_pos_lap_ids = included_pos_lap_ids[np.isin(included_pos_lap_ids, sess.laps.lap_id)]
    # included_pos_lap_ids

    # print(f'included_pos_lap_ids: {included_pos_lap_ids}')

    # lap_grouped_position_df = curr_position_df.groupby('lap')
    # lap_specific_position_dfs = [lap_grouped_position_df.get_group(i)[['t','aclu','x','y','lin_pos']] for i in included_lap_ids] # dataframes split for each ID:

    for i, curr_cell_ID in enumerate(included_cell_IDs):
        plot_data_dict = _plot_single_spikes(p, cell_specific_spikes_dfs, i)


# sess = curr_kdiba_pipeline.filtered_sessions['maze']
# included_cell_IDXs = [6]
# included_lap_IDXs = [2, 3]
# _plot_all_lap_spikes(pActiveInteractiveLapsPlotter, sess, included_cell_IDXs, included_lap_IDXs)

from PhoGui.InteractivePlotter.InteractiveCustomDataExplorer import InteractiveCustomDataExplorer

lap_start_z = 0.0
# lap_id_dependent_z_offset = 0.0
lap_id_dependent_z_offset = 3.0
# curr_kdiba_pipeline.active_configs['maze1'].lap_id_dependent_z_offset = 3.0

def _display_testing(sess, computation_result, active_config, extant_plotter=None):
    """ Testing of plot_lap_trajectories_2d """
    # from PhoPositionalData.plotting.laps import plot_lap_trajectories_2d
    # fig, axs, laps_pages = plot_lap_trajectories_2d(sess, curr_num_subplots=len(sess.laps.lap_id), active_page_index=0)
    # fig.suptitle('Lap Trajectories 2D', fontsize=22)
    # # fig_out_path = active_config.plotting_config.get_figure_save_path('lap_trajectories_2D').with_suffix('.png')
    # # fig.savefig(fig_out_path)
    # # plt.show()

    # active_laps_config = InteractivePlaceCellConfig(active_session_config=active_config.active_session_config, active_epochs=None, video_output_config=None, plotting_config=None) # '3|1    
    
    # active_laps_config = active_config
    # active_laps_config.plotting_config = PlottingConfig(output_subplots_shape='1|5', output_parent_dir=Path('output', sess.config.session_name, 'custom_laps'))
    print(f'active_config.plotting_config: {active_config.plotting_config}')
    # active_config.plotting_config = PlottingConfig(output_subplots_shape='1|5', output_parent_dir=Path('output', sess.config.session_name, 'custom_laps'))
    
    single_combined_plot=True
    if single_combined_plot:
        default_plotting = True
    else:
        default_plotting = False
    
    # if active_config.plotting_config.plotter_type is None:
    active_config.plotting_config.plotter_type = 'MultiPlotter'
    print(f'active_config.plotting_config: {active_config.plotting_config}')
    iplapsDataExplorer = InteractiveCustomDataExplorer(active_config, sess, extant_plotter=extant_plotter)
    pActiveInteractiveLapsPlotter = iplapsDataExplorer.plot(pActivePlotter=extant_plotter, default_plotting=default_plotting)

    # included_cell_idxs = None
    included_cell_idxs = [0, 1]
    # included_lap_idxs = [2, 5, 9, 12]
    included_lap_idxs = [2]
    
    # All
    # included_cell_idxs = np.arange(len(sess.spikes_df.spikes.neuron_ids))
    # included_lap_idxs = np.arange(len(sess.laps.lap_id))
    
    from PhoPositionalData.plotting.laps import plot_lap_trajectories_3d
    pActiveInteractiveLapsPlotter, laps_pages = plot_lap_trajectories_3d(sess, curr_num_subplots=5, active_page_index=0, included_lap_idxs=included_lap_idxs, single_combined_plot=single_combined_plot, 
                                                                         lap_start_z = lap_start_z, lap_id_dependent_z_offset = lap_id_dependent_z_offset,
                                                                         existing_plotter=pActiveInteractiveLapsPlotter)
    

    # add the spikes for the curves:
    _plot_all_lap_spikes(pActiveInteractiveLapsPlotter, sess, included_cell_idxs, included_lap_idxs, lap_start_z=lap_start_z, lap_id_dependent_z_offset=lap_id_dependent_z_offset)


    return iplapsDataExplorer, pActiveInteractiveLapsPlotter



# curr_kdiba_pipeline.computation_results['maze1'].computation_config
# curr_kdiba_pipeline.computation_results['maze1'].sess.config
# curr_kdiba_pipeline.active_configs['maze1']

curr_result_label = 'maze1'
sess = curr_kdiba_pipeline.filtered_sessions[curr_result_label]
sess = curr_kdiba_pipeline.sess

pActiveInteractiveLapsPlotter = None
try: pActiveInteractiveLapsPlotter
except NameError: pActiveInteractiveLapsPlotter = None # Checks variable p's existance, and sets its value to None if it doesn't exist so it can be checked in the next step
iplapsDataExplorer, pActiveInteractiveLapsPlotter = _display_testing(sess, curr_kdiba_pipeline.computation_results[curr_result_label], curr_kdiba_pipeline.active_configs[curr_result_label],
                                                                     extant_plotter=pActiveInteractiveLapsPlotter)
pActiveInteractiveLapsPlotter.show()

In [ ]:
# lines_from_points(pdata.points)


np.shape(all_points)

In [ ]:
p, laps_pages = _plot_lap_trajectories_combined_plot_3d(curr_kdiba_pipeline.sess, curr_num_subplots=5, single_combined_plot=False)
p.show()